# Caravan and Camping Club POI Generator 2021

In [1]:
# Install any missing libraries
!pip install simplekml
!pip install pandas
!pip install folium

In [1]:
# imports
import pandas as pd
import simplekml
import folium

In [4]:
df = pd.read_csv("data/Camping-in-the-Forest.csv",names=['Longitude','Latitude','Address'], header=0, sep=',')
print(df.head(5))
print(df['Address'].head())

   Longitude   Latitude                                            Address
0  -1.530129  50.889452       "Ashurst  New Forest SO40 7AR>02380 292097""
1  -2.624735  51.812785  "Bracelands  Forest of Dean GL16 7NP>01594 837...
2  -4.581170  56.112450        "Cashel  Loch Lomond G63 0AW>01360 870234""
3  -4.366124  56.158363       "Cobleland  Trossachs FK8 3RR>01877 382392""
4  -1.526756  50.861170    "Denny Wood  New Forest SO43 7FZ>02380 293144""
0         "Ashurst  New Forest SO40 7AR>02380 292097""
1    "Bracelands  Forest of Dean GL16 7NP>01594 837...
2          "Cashel  Loch Lomond G63 0AW>01360 870234""
3         "Cobleland  Trossachs FK8 3RR>01877 382392""
4      "Denny Wood  New Forest SO43 7FZ>02380 293144""
Name: Address, dtype: object


### Extract Site Name, Telephone Number, and Address in to their own columns

In [8]:
temp = df['Address'].apply(lambda x: pd.Series(str(x).split('>')))
print(temp)  # check if data is okay, one line had less quotes in it for some reason!!

df["PhoneNumber"]= temp[1]
df['PhoneNumber'] = df['PhoneNumber'].astype(str)

# Remove extra quotes the camping club add to the end of each line!
df['PhoneNumber'] = df.apply(lambda x: x['PhoneNumber'][:-2], axis=1)

# making separate phone number column from new data frame 
#temp_address = temp[0].apply(lambda x: pd.Series(str(x).split(' - ')))
#df["Town"] = temp_address[0]
#df["SiteName"] = temp_address[1]

#df["Town"] = df.apply(lambda x: x['Town'][1:], axis=1)

# Dropping old Address columns 
df.drop(columns =["Address"], inplace = True) 

df 

                                       0               1
0          "Ashurst  New Forest SO40 7AR  02380 292097""
1   "Bracelands  Forest of Dean GL16 7NP  01594 837258""
2           "Cashel  Loch Lomond G63 0AW  01360 870234""
3          "Cobleland  Trossachs FK8 3RR  01877 382392""
4       "Denny Wood  New Forest SO43 7FZ  02380 293144""
5         "Glenmore  Cairngorms PH22 1QU  01479 861271""
6    "Hollands Wood  New Forest SO42 7QH  01590 622967""
7         "Holmsley  New Forest BH23 8EB  01425 674502""
8        "Longbeech  New Forest SO43 7HH  02380 812740""
9      "Matley Wood  New Forest SO43 7FZ  02380 293144""
10         "Ocknell  New Forest SO43 7HH  02380 812740""
11      "Postern Hill  Wiltshire SN8 4ND  01672 515195""
12       "Roundhill  New Forest SO42 7QL  01590 624344""
13       "Setthorns  New Forest BH25 5WA  01590 681020""


,Longitude,Latitude,PhoneNumber,Town
0,-1.530129,50.889452,02380 292097,"""Ashurst New Forest SO40 7AR"
1,-2.624735,51.812785,01594 837258,"""Bracelands Forest of Dean GL16 7NP"
2,-4.581170,56.112450,01360 870234,"""Cashel Loch Lomond G63 0AW"
3,-4.366124,56.158363,01877 382392,"""Cobleland Trossachs FK8 3RR"
4,-1.526756,50.861170,02380 293144,"""Denny Wood New Forest SO43 7FZ"
5,-3.694239,57.167368,01479 861271,"""Glenmore Cairngorms PH22 1QU"
6,-1.571802,50.829241,01590 622967,"""Hollands Wood New Forest SO42 7QH"
7,-1.705375,50.792885,01425 674502,"""Holmsley New Forest BH23 8EB"
8,-1.649765,50.914578,02380 812740,"""Longbeech New Forest SO43 7HH"
9,-1.534645,50.865636,02380 293144,"""Matley Wood New Forest SO43 7FZ"


### Now create kml file for use by maps.me

### Once the .kml file has been created for maps.me will need to edit the file and change all instances of <styleUrl>#<number></styleUrl> with a valid maps.me styleUrl
    
<styleUrl>#placemark-blue</styleUrl>
<styleUrl>#placemark-red</styleUrl>
<styleUrl>#placemark-green</styleUrl>  
<styleUrl>#placemark-yellow</styleUrl>
<styleUrl>#placemark-brown</styleUrl>
<styleUrl>#placemark-orange</styleUrl>
<styleUrl>#placemark-pink</styleUrl>    
<styleUrl>#placemark-purple</styleUrl> *

In [12]:
kml = simplekml.Kml()
style = simplekml.Style()
style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/pink-blank.png'
style.iconstyle.scale = 1

map = folium.Map(location=[54.0, -1.0], zoom_start=6)
for index, row in df.iterrows():
    #name = row['SiteName'] + ', ' + row['SiteAddress'] + ', ' + row['PhoneNumber']
    #name = row['SiteName']
    #address = row['Town'] + ', ' + row['SiteName'] + ', ' + row['PhoneNumber']
    name = row['Town'] 
    address = row['Town'] + ', ' + row['PhoneNumber']
    phonenumber = row['PhoneNumber']
    coords = [(row['Longitude'], row['Latitude'])]
    pnt = kml.newpoint(name=name)
    pnt.coords = coords
    pnt.style = style
    pnt.phonenumber = phonenumber
    pnt.description = address
    
    popupText = folium.Html('<p>'+row['Town']+'</p><p>'+'</p><p>'+row['PhoneNumber']+'</p>', script=True)
    #popupText = folium.Html('<p>'+row['SiteName']+'</p><p>'+row['SiteAddress']+'</p>', script=True)
    popup = folium.Popup(popupText, max_width=300)
    folium.Marker([row['Latitude'], row['Longitude']],
                 popup=popup,
                 icon=folium.Icon(color='pink', icon='info-sign')).add_to(map)
    
kml.save(path="poi/ccc-forest-2021.kml")
map.save("poi/ccc-forest-2021.html")
map

In [46]:
#!pip list